In [2]:
from stanfordcorenlp import StanfordCoreNLP as stnlp
nlp = stnlp(r'/home/ayush/stanford-corenlp-full-2018-01-31')

In [3]:
def getMapping(sentence):
    open_bracket = []
    mapping = {}
    for i,j in enumerate(sentence):
        if(j=='('):
            open_bracket.append(i)
        elif (j==')'):
            mapping[open_bracket[-1]] = i
            del open_bracket[-1]
    return mapping

In [4]:
#OUTPUT in form of [NP[VP[NP[VP[NP...]]]]]
def SVO(sentence):
    sentence = nlp.parse(sentence)
    sentence = str(sentence)
    ar = [a.strip() for a in sentence.split('\n')]
    sentence = ''.join(ar)
    return getTags(sentence)

def getTags(sentence):
    mapping = getMapping(sentence)
    if '(NP' in sentence:
        if '(VP' in sentence:
            if sentence.index('(NP')<sentence.index('(VP'):
                return getNP(sentence, 0, mapping)
            else:
                return getVP(sentence, 0, mapping)
        else:
            return getNP(sentence, 0, mapping)
    elif '(VP' in sentence:
        return getVP(sentence, 0, mapping)
    
def getNP(sentence, idxStart, mapping):
    res = []
    idxNP = sentence.index('(NP')
    idxNPend = mapping[idxStart + idxNP] - idxStart
    subj = sentence[idxNP:idxNPend]
    npAdded = False
    
    if '(VP' in sentence[idxNP:idxNPend]:
        #NP(VP)
        remain = sentence[idxNP:idxNPend]
        verblist = getVP(remain, idxNP+idxStart, mapping)
        if type(verblist) == type('str'):
            temp = verblist
        else:
            temp = verblist[0]
        while (type(temp)!=type('str')):
            temp = temp[0]
        if temp in subj:
            idxx = subj.index(temp)
            subj = subj[:idxx]
        res.append(subj)
        res.append(verblist)
        npAdded = True    
    
    remain = sentence[idxNPend+1:]
    if '(VP' in remain:
        #NP-VP
        verblist = getVP(remain, idxNPend+1+idxStart, mapping)
        if not npAdded:
            res.append(subj)
        res.append(verblist)
    else:
        #NP
        if not subj in res:
            res.append(subj) 
    return res
        
def getVP(sentence, idxStart, mapping):
    res = []
    idxVP = sentence.index('(VP')
    idxVPend = mapping[idxStart + idxVP] - idxStart
    verb = sentence[idxVP:idxVPend]
    
    verbAdded = False
    if '(NP' in sentence[idxVP:idxVPend]:
        remain = sentence[idxVP:idxVPend]
        nplist = getNP(remain, idxVP+idxStart, mapping)
        if type(nplist) == type('str'):
            temp = nplist
        else:
            temp = nplist[0]
        while (type(temp)!=type('str')):
            temp = temp[0]
        if temp in verb:
            idxx = verb.index(temp)
            verb = verb[:idxx]
        res.append(verb)
        res.append(nplist)
        verbAdded = True
    
    remain = sentence[idxVPend+1:]
    if '(NP' in remain:
        nplist = getNP(remain, idxVPend+1+idxStart, mapping)
        if not verbAdded:
            res.append(verb)
        res.append(nplist)
    else:
        if not verb in res:
            res.append(verb)
    return res

In [5]:
sen =  "Architecturally, the school has a Catholic character. Atop the Main Building gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend Venite Ad Me Omnes. Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive, is a simple, modern stone statue of Mary."
sen = sen.split('.')
del sen[-1]
sen

['Architecturally, the school has a Catholic character',
 ' Atop the Main Building gold dome is a golden statue of the Virgin Mary',
 ' Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend Venite Ad Me Omnes',
 ' Next to the Main Building is the Basilica of the Sacred Heart',
 ' Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection',
 ' It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858',
 ' At the end of the main drive, is a simple, modern stone statue of Mary']

In [6]:
for s in sen:
    print SVO(s)
    print '----------------------------------------------------------------------------------'

['(NP (DT the) (NN school)', ['(VP (VBZ has)', ['(NP (DT a) (JJ Catholic) (NN character)']]]
----------------------------------------------------------------------------------
['(NP (DT the) (NNP Main) (NNP Building) (NN gold)', ['(VP (VBZ is)', ['(NP(NP (DT a) (JJ golden) (NN statue))(PP (IN of)(NP (DT the) (NNP Virgin) (NNP Mary)))']]]
----------------------------------------------------------------------------------
['(NP(NP (NN front))(PP (IN of)(NP (DT the) (NNP Main) (NNP Building)))', ['(VP (VBG facing)(SBAR(S', ['(NP (PRP it)', ['(VP (VBZ is)', ['(NP(NP (DT a) (NN copper) (NN statue))(PP (IN of)(NP(NP (NNP Christ))(PP (IN with)(NP(NP (NNS arms))', ['(VP (VBN upraised)(PP (IN with)', ['(NP(NP (DT the) (NN legend) (JJ Venite) (NN Ad))(SBAR(S(NP (PRP Me))', ['(VP (VBZ Omnes)']]]]]]]]
----------------------------------------------------------------------------------
['(NP (DT the) (NNP Main) (NNP Building)', ['(VP (VBZ is)', ['(NP(NP (DT the) (NN Basilica))(PP (IN of)(NP (DT the) (

In [ ]:
import regex
sen =  "Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."
ner = 
sen = sen.split('.')
del sen[-1]

stop_words = ['a', 'an', 'the']
replace = ["\"", "\'s", "\'"]
for i in range(len(sen)):
    sen[i] = regex.sub(r' \(.*\)', '', sen[i])
    sen[i] = regex.sub(r'\(.*\)', '', sen[i])
    for r in replace:
        sen[i] = sen[i].replace(r, "")
    sen[i] = " ".join([s for s in sen[i].split(" ") if s not in stop_words])
    sen[i] = " ".join(sen[i].split())